## Jordi Blasco Lozano 
## 14. Grafos en la vida real

### 14.1 Node2Vec

Node2Vec es un algoritmo de representación de un grafo en un espacio de dimensión reducida. La idea es que nodos que están cerca en el grafo original, estén cerca en el espacio de dimensión reducida. Para ello, se utiliza un algoritmo de random walk sesgado que permite recorrer el grafo y obtener las representaciones de los nodos en el espacio de dimensión reducida.

### 14.1.1. Incrustaciones y word2vec 

- Las incrustaciones de los nodos en un gráfico son las representaciones de baja dimensión de los nodos en el gráfico. 
- Las incrustaciones de los nodos se pueden utilizar para realizar diversas tareas en el gráfico, como clasificación, agrupación y predicción de enlaces. 
- El algoritmo Node2Vec se basa en el algoritmo Word2Vec. Word2Vec es un algoritmo de aprendizaje de representación que se utiliza para aprender las incrustaciones de palabras en un texto.